<a href="https://colab.research.google.com/github/davemlz/eemont/blob/master/tutorials/006-NDSI-and-Snow-Cover-Sentinel-2-MOD10A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Overloaded Operators to Compute Snow Cover (Sentinel-2, MOD10A1)

## Let's start!

If required, please uncomment:

In [1]:
!pip install earthengine-api
!pip install eemont
!pip install geemap

  Created wheel for eemont: filename=eemont-0.1.7-cp36-none-any.whl size=16387 sha256=b5ce2073f5f9e1deffdb5699987065ce468f36baabe1a91e8c378633de09886d
  Stored in directory: /root/.cache/pip/wheels/ca/ba/9a/3fc1205f53a16e573daef50a4db1d759940ad173acca050589
Successfully built eemont
     |████████████████████████████████| 399kB 8.0MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 1.9MB 13.1MB/s 
     |████████████████████████████████| 3.3MB 40.0MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 5.1MB 52.9MB/s 
     |████████████████████████████████| 153kB 37.0MB/s 
     |████████████████████████████████| 1.1MB 39.7MB/s 
     |████████████████████████████████| 225kB 48.1MB/s 
     |████████████████████████████████| 122kB 23.0MB/s 
     |████████████████████████████████| 552kB 38.4MB/s 
     |████████████████████████████████| 194kB 42.

Import the required packges.

In [2]:
import ee, eemont, geemap.eefolium as geemap

If required (e.g. in Google Colab), please uncomment:

In [3]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=rJePxUq0N-iakxUEVwDXD9WNDnrmAMPjP536_86kmi4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4EYz0zKmWkyuCVIomOlimmnEjNlG8-R4NJrX-0_qJ-o5KPF4u7F9E

Successfully saved authorization token.


Initialize Google Earth Engine.

In [4]:
ee.Initialize()

Get, filter, mask clouds and scale the Sentinel-2 image collection.

In [5]:
point = ee.Geometry.Point([-76.0269,2.92846])

In [6]:
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
      .maskClouds()
      .scale()
      .index('NDSI')) # Let's compute the NDSI, we'll need it!

Let's select the required bands:

In [8]:
NDSI = S2.select('NDSI')
N = S2.select('B8')
G = S2.select('B3')

## Overloaded Operators

`eemont` has overloaded the binary operators, rich comparisons and unary operators in the following list for the `ee.Image` class:

(+, -, \*\, /, //, %, \**\, <<, >>, &, |, <, <=, ==, !=, >, >=, -, ~)

Therefore, you can now use them for image operations!

The following line computes snow cover according to [(Hall et al., 2001)](https://modis.gsfc.nasa.gov/data/atbd/atbd_mod10.pdf):

In [9]:
snowPixels = (NDSI > 0.4) & (N >= 0.1) & (G > 0.11) # Overloaded operators: >, >=, &.

Now, update the mask of the NDSI.

In [10]:
NDSI = NDSI.updateMask(snowPixels)

Let's save the date of the image to get the closest MOD10A1 image for comparison:

In [11]:
dateOfInterest = ee.Date(S2.get('system:time_start'))

Get, filter and scale the MOD10A1 product:

In [12]:
MOD10A1 = (ee.ImageCollection('MODIS/006/MOD10A1')
           .filterBounds(point)
           .closest(dateOfInterest)
           .scale() # NEW! Note that the scale() method supports the MOD10A1 product!
           .first())

This product already has the snow cover, therefore, we just need to compute the NDSI operation for snow cover pixels (Let's use overloaded operators!):

In [13]:
NDSI_MODIS = MOD10A1.select('NDSI')
NDSI_MODIS = NDSI_MODIS.updateMask(NDSI_MODIS > 0.4) # The overloaded operator > is used here!

## Visualization

Let's define the NDSI visualization parameters:

In [14]:
visNDSI = {
    'min':0.4,
    'max':1,
    'palette': ['000000', '0dffff', '0524ff', 'ffffff']
}

Let's define the RGB visualization parameters:

In [15]:
visRGB = {
    'min':0,
    'max':0.3,
    'bands':['B4', 'B3', 'B2']
}

Use `geemap` to display results:

In [16]:
Map = geemap.Map()
Map.addLayer(S2,visRGB,'Sentinel-2 RGB')
Map.addLayer(NDSI_MODIS,visNDSI,'MODIS NDSI')
Map.addLayer(NDSI,visNDSI,'Sentinel-2 NDSI')
Map.add_colorbar(visNDSI['palette'],caption = 'NDSI')
Map.centerObject(point,13)
Map.addLayerControl()
Map